In [1]:
# Inspect GPU
!nvidia-smi

Wed May  5 08:06:32 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import files
files.upload()

In [20]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [22]:
save_location =  "./drive/MyDrive/11785/hw4p2/"

In [5]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [6]:
!kaggle competitions download -c 11785-spring2021-hw4p2

  0% 0.00/3.76M [00:00<?, ?B/s]
100% 3.76M/3.76M [00:00<00:00, 62.4MB/s]
  0% 0.00/784k [00:00<?, ?B/s]
100% 784k/784k [00:00<00:00, 52.3MB/s]
  0% 0.00/112k [00:00<?, ?B/s]
100% 112k/112k [00:00<00:00, 36.3MB/s]
100% 3.35G/3.36G [03:23<00:00, 24.7MB/s]
100% 3.36G/3.36G [03:24<00:00, 17.7MB/s]
 99% 181M/183M [00:22<00:00, 6.94MB/s]
100% 183M/183M [00:22<00:00, 8.41MB/s]
 99% 179M/182M [00:18<00:00, 17.3MB/s]
100% 182M/182M [00:18<00:00, 10.1MB/s]


In [7]:
!unzip dev.npy.zip
!unzip test.npy.zip
!unzip train.npy.zip
!unzip train_transcripts.npy.zip
!unzip dev_transcripts.npy.zip

Archive:  dev.npy.zip
  inflating: dev.npy                 
Archive:  test.npy.zip
  inflating: test.npy                
Archive:  train.npy.zip
  inflating: train.npy               
Archive:  train_transcripts.npy.zip
  inflating: train_transcripts.npy   
unzip:  cannot find or open dev_transcripts.npy.zip, dev_transcripts.npy.zip.zip or dev_transcripts.npy.zip.ZIP.


In [8]:
!pip install python-Levenshtein

     |████████████████████████████████| 51kB 4.0MB/s 
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.2-cp37-cp37m-linux_x86_64.whl size=149798 sha256=3e4bb8cf16122d09544cbe74482bdcee160143d175a66032880247b6863561c9
  Stored in directory: /root/.cache/pip/wheels/b3/26/73/4b48503bac73f01cf18e52cd250947049a7f339e940c5df8fc
Successfully built python-Levenshtein


In [1]:
import sys
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.utils.rnn as rnn_utils
import torch.optim as optim
import torch.nn.utils as utils
import seaborn as sns
import matplotlib.pyplot as plt
import time
import random
from torch.utils import data
import Levenshtein

cuda = torch.cuda.is_available()
print(cuda, sys.version)
device = torch.device("cuda" if cuda else "cpu")
np.random.seed(5111785)
torch.manual_seed(5111785)

LETTER_LIST = ['<sos>', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', \
         'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '-', "'", '.', '_', '+', ' ', '<eos>']

True 3.7.10 (default, Feb 20 2021, 21:17:23) 
[GCC 7.5.0]


In [2]:
def create_dictionaries(letter_list):
    '''
    Create dictionaries for letter2index and index2letter transformations
    '''
    letter2index = {letter:index for index, letter in enumerate(letter_list)}
    index2letter = {index:letter for index, letter in enumerate(letter_list)}
    return letter2index, index2letter
    
def transform_letter_to_index(raw_transcripts):
    '''
    Transforms text input to numerical input by converting each letter 
    to its corresponding index from letter_list

    Args:
        raw_transcripts: Raw text transcripts with the shape of (N, )
    
    Return:
        transcripts: Converted index-format transcripts. This would be a list with a length of N
    '''  
    index_transcript = []
    for sentence in raw_transcripts:
      try:
        sentence = " ".join([word.decode("utf-8") for word in sentence])
      except:
        sentence = " ".join([word for word in sentence])
      index_transcript.append([letter2index[LETTER_LIST[0]]]+[letter2index[letter] for letter in sentence]+[letter2index[LETTER_LIST[-1]]])
    return index_transcript

def transform_index_to_letter(index_transcript):
    letter_transcript = []
    for idx in index_transcript:
      letter_transcript.append(
          ''.join(
              [LETTER_LIST[int(idx[i])] for i in range(len(idx))]
              )
          )
    
    return letter_transcript
# Create the letter2index and index2letter dictionary
letter2index, index2letter = create_dictionaries(LETTER_LIST)

In [3]:
# Load the training, validation and testing data
train_data = np.load('train.npy', allow_pickle=True, encoding='bytes')
valid_data = np.load('dev.npy', allow_pickle=True, encoding='bytes')
# test_data = np.load('test.npy', allow_pickle=True, encoding='bytes')

# Load the training, validation raw text transcripts
raw_train_transcript = np.load('train_transcripts.npy', allow_pickle=True,encoding='bytes')
raw_valid_transcript = np.load('dev_transcripts.npy', allow_pickle=True,encoding='bytes')

# TODO: Convert the raw text transcripts into indexes
train_transcript = transform_letter_to_index(raw_train_transcript)
valid_transcript = transform_letter_to_index(raw_valid_transcript)

In [4]:
# train_transcript[0]

In [5]:
# raw_train_transcript

In [6]:
class MyDataset(data.Dataset):
    def __init__(self, X, Y=None):
        self.X = X
        self.Y = Y

    def __len__(self):
        return self.X.shape[0]

    def __getitem__(self, index):
        # For testing set, return only x
        if self.Y == None:
            return torch.tensor(self.X[index].astype(np.float32))
        # For training and validation set, return x and y
        else:
            return torch.tensor(self.X[index].astype(np.float32)), torch.tensor(self.Y[index])

def collate_train_val(data):
    """
    Return:
        pad_x: the padded x (training/validation speech data) 
        pad_y: the padded y (text labels - transcripts)
        x_len: the length of x
        y_len: the length of y
    """
    Xs, ys, x_lengths, y_lengths = [], [], [], []

    for x,y in data:
        Xs.append(x)
        ys.append(y)
        x_lengths.append(x.shape[0])
        y_lengths.append(len(y))
    return (rnn_utils.pad_sequence(Xs, batch_first=False), x_lengths), (rnn_utils.pad_sequence(ys, batch_first=True), y_lengths)

def collate_test(data):
    """
    Return:
        pad_x: the padded x (testing speech data) 
        x_len: the length of x
    """
    Xs, x_lengths = [], []

    for x in data:
      Xs.append(x)
      x_lengths.append(x.shape[0])

    return (rnn_utils.pad_sequence(Xs, batch_first=False), x_lengths)

In [7]:
BATCH_SIZE = 64

In [8]:
# Create datasets
train_dataset = MyDataset(train_data, train_transcript)
valid_dataset = MyDataset(valid_data, valid_transcript)
# test_dataset = MyDataset(test_data)

# Create data loaders
train_loader = data.DataLoader(train_dataset, batch_size
                               =BATCH_SIZE, shuffle=True, 
                               num_workers=4, pin_memory=True, collate_fn=collate_train_val)
valid_loader = data.DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=False, 
                               num_workers=4, pin_memory=True, collate_fn=collate_train_val)

In [9]:
class pBLSTM(nn.Module):
    '''
    Pyramidal BiLSTM
    Read paper and understand the concepts and then write your implementation here.
    '''
    def __init__(self, input_dim, hidden_dim):
        super(pBLSTM, self).__init__()
        self.blstm = nn.LSTM(input_size=input_dim, hidden_size=hidden_dim, num_layers=1, bidirectional=True, batch_first=True)

    def forward(self, x, lens):
      x, hidden_state = self.blstm(x)
      #print("input >>>>>",x.shape)
      x, output_lens = rnn_utils.pad_packed_sequence(x, batch_first=True)
      #print("shape:", x.shape)
      if x.shape[1] % 2 != 0:
        x = x[:,:-1,:]
        #print("shape:", x.shape)
      x = x.reshape(x.shape[0], x.shape[1]//2, x.shape[2]*2)
      #print("after reshape:", x.shape)
      out = rnn_utils.pack_padded_sequence(x, output_lens//2, enforce_sorted=False, batch_first=True) # repack for next layer
      # print("out shape:", out.shape)
      return out, np.array(output_lens)//2

In [10]:
class Encoder(nn.Module):
    '''
    Encoder takes the utterances as inputs and returns the key, value and unpacked_x_len.
    Key and value are linear projections of the output from pBLSTM network for the laster.
    '''
    def __init__(self, input_dim, encoder_hidden_dim, key_value_size=128):
        super(Encoder, self).__init__()
        # The first LSTM at the very bottom
        self.lstm = nn.LSTM(input_size=input_dim, hidden_size=encoder_hidden_dim, num_layers=1, bidirectional=True, batch_first=True)


        # TODO: Define the blocks of pBLSTMs
        # ...
        self.pBiLSTM1 = pBLSTM(2*encoder_hidden_dim, encoder_hidden_dim//2)
        self.pBiLSTM2 = pBLSTM(2*encoder_hidden_dim, encoder_hidden_dim//2)
        self.pBiLSTM3 = pBLSTM(2*encoder_hidden_dim, encoder_hidden_dim//2)
        # The linear transformation for producing Key and Value for attention
        # Since you are using bidirectional LSTM, be careful about the size of hidden dimension
        self.key_network = nn.Linear(encoder_hidden_dim*2, key_value_size)
        self.value_network = nn.Linear(encoder_hidden_dim*2, key_value_size)

    def forward(self, x, x_len):
      # print(x_len)
      # print(x.shape)
      # Pass through the first LSTM at the very bottom
      packed_sequence = rnn_utils.pack_padded_sequence(x, x_len, enforce_sorted=False, batch_first=False) 
      packed_out, _ = self.lstm(packed_sequence)
      # unpacked, _ = rnn_utils.pad_packed_sequence(packed_out)
      

      # TODO: Pass through the pBLSTM blocks
      # ...
      layer_out, new_len = self.pBiLSTM1(packed_out, x_len)
      layer_out, new_len = self.pBiLSTM2(layer_out, new_len)
      layer_out, new_len = self.pBiLSTM3(layer_out, new_len)
      # Unpack the sequence and get the Key and Value for attention
      pblstm_out, unpacked_x_len = rnn_utils.pad_packed_sequence(layer_out, batch_first=True)
      
      keys = self.key_network(pblstm_out)
      values = self.value_network(pblstm_out)
      return keys, values, unpacked_x_len

In [11]:
def plot_attention(attention):
    plt.clf()
    sns.heatmap(attention, cmap='hot')
    plt.show()

class Attention(nn.Module):
    '''
    Attention is calculated using key, value and query from Encoder and decoder.
    Below are the set of operations you need to perform for computing attention:
        energy = bmm(key, query)
        attention = softmax(energy)
        context = bmm(attention, value)
    '''
    def __init__(self):
        super(Attention, self).__init__()

    def forward(self, query, key, value, mask):
        # Compute (batch_size, max_len) attention logits. "bmm" stands for "batch matrix multiplication".
        # Input shape of bmm:  (batch_szie, max_len, hidden_size), (batch_size, hidden_size, 1) 
        # Output shape of bmm: (batch_size, max_len, 1)
        energy = torch.bmm(key, query.unsqueeze(2)).squeeze(2)
        
        # Create an (batch_size, max_len) boolean mask for all padding positions
        # Make use of broadcasting: (1, max_len), (batch_size, 1) -> (batch_size, max_len)
        mask_lengths = torch.arange(key.size(1)).unsqueeze(0) >= mask.unsqueeze(1)
        
        # Set attention logits at padding positions to negative infinity.
        energy.masked_fill_(mask_lengths.to(device), -1e9)
        
        # Take softmax over the "source length" dimension.
        attention = nn.functional.softmax(energy, dim=1)
        
        # Compute attention-weighted sum of context vectors
        # Input shape of bmm: (batch_size, 1, max_len), (batch_size, max_len, hidden_size) 
        # Output shape of bmm: (batch_size, 1, hidden_size)
        context = torch.bmm(attention.unsqueeze(1), value).squeeze(1)
        
        # attention vectors are returned for visualization
        return context, attention

In [12]:
class Decoder(nn.Module):
    '''
    As mentioned in a previous recitation, each forward call of decoder deals with just one time step.
    Thus we use LSTMCell instead of LSTM here.
    The output from the seond LSTMCell can be used as query for calculating attention.
    In place of value that we get from the attention, this can be replace by context we get from the attention.
    Methods like Gumble noise and teacher forcing can also be incorporated for improving the performance.
    '''
    def __init__(self, vocab_size, decoder_hidden_dim, embed_dim, key_value_size=128):
        super(Decoder, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=letter2index['<eos>'])
        self.lstm1 = nn.LSTMCell(input_size=embed_dim + key_value_size, hidden_size=decoder_hidden_dim)
        self.lstm2 = nn.LSTMCell(input_size=decoder_hidden_dim, hidden_size=key_value_size)
      
        self.attention = Attention()     
        self.vocab_size = vocab_size
        self.character_prob = nn.Linear(2 * key_value_size, vocab_size)
        self.key_value_size = key_value_size

    def forward(self, key, value, encoder_len, y=None, mode='train', teacher_force_prob=0.3):
        '''
        Args:
            key :(B, T, key_value_size) - Output of the Encoder Key projection layer
            value: (B, T, key_value_size) - Output of the Encoder Value projection layer
            y: (T, text_len) - Batch input of text with text_length
            mode: Train or eval mode
        Return:
            predictions: the character perdiction probability 
        '''

        B, key_seq_max_len, key_value_size = key.shape


        if mode == 'train':
            max_len =  y.shape[1]
            char_embeddings = self.embedding(y)
        else:
            max_len = 600

        # TODO: Create the attention mask here (outside the for loop rather than inside) to aviod repetition
        # ...
        attention_mask = []
        
        predictions = []
        prediction = torch.zeros(B, 1).to(device)
        hidden_states = [None, None] 
        
        # TODO: Initialize the context. Be careful here
        context = value[:, 0, :]
        
        for i in range(max_len):
            if mode == 'train':
                # TODO: Implement (1) Teacher Forcing and (2) Gumble Noise techniques here
                # ...
                # teacher forcing
                if np.random.random() < teacher_force_prob:
                  char_embed = char_embeddings[:,i,:]
                else:
                  prediction = F.gumbel_softmax(prediction, tau=1)
                  char_embed = self.embedding(prediction.argmax(dim=-1))
            else:
                char_embed = self.embedding(prediction.argmax(dim=-1))

            y_context = torch.cat([char_embed, context], dim=1)
            hidden_states[0] = self.lstm1(y_context, hidden_states[0])

            lstm1_hidden = hidden_states[0][0]
            hidden_states[1] = self.lstm2(lstm1_hidden, hidden_states[1])
            output = hidden_states[1][0]
            
            # TODO: Compute attention from the output of the second LSTM Cell
            # ...
            context, attention = self.attention(output,key, value, encoder_len)
            
            output_context = torch.cat([output, context], dim=1)
            prediction = self.character_prob(output_context)
            predictions.append(prediction.unsqueeze(1))
            
            attention_mask.append(attention.detach().cpu().numpy())
        attention_mask = np.array(attention_mask)

        mask = attention_mask[:,0, :encoder_len[0]]
        return torch.cat(predictions, dim=1), mask

In [13]:
class Seq2Seq(nn.Module):
    '''
    We train an end-to-end sequence to sequence model comprising of Encoder and Decoder.
    This is simply a wrapper "model" for your encoder and decoder.
    '''
    def __init__(self, input_dim, vocab_size, encoder_hidden_dim, decoder_hidden_dim, embed_dim, key_value_size=128):
        super(Seq2Seq,self).__init__()
        self.encoder = Encoder(input_dim, encoder_hidden_dim, key_value_size=key_value_size)
        self.decoder = Decoder(vocab_size, decoder_hidden_dim, embed_dim, key_value_size=key_value_size)

    def forward(self, x, x_len, y=None, mode='train'):
        key, value, encoder_len = self.encoder(x, x_len)
        predictions, attention = self.decoder(key, value, encoder_len, y=y, mode=mode)
        return predictions, attention

In [14]:
def train(model, train_loader, criterion, optimizer, mode):
    model.train()
    running_loss = 0
    
    # 0) Iterate through your data loader
    for batch, (X, y) in enumerate(train_loader):
      optimizer.zero_grad()
      # 1) Set the inputs to the device.
      X_data, X_len = X
      X_data = X_data.to(device)
      # X_len = X_len.to(device)
      y_data, y_len = y
      y_data = y_data.to(device)
      y_data_in = y_data[:,:-1]
      y_data_out = y_data[:,1:]
      #print("targets>>>",y_data_out.shape)
      #print("y_data",y_data_out.shape)

      # 2) Pass your inputs, and length of speech into the model.
      predictions, att = model(X_data, X_len, y_data_in)
      #print("pred>>>",predictions.shape)
      # 3) Generate a mask based on the lengths of the text
      #    Ensure the mask is on the device and is the correct shape. 
      mask = torch.Tensor(np.zeros(y_data_in.shape)).to(device)
      for i in range(len(y_len)):
        mask[i, :y_len[i]] = 1
      mask.to(device)
      mask = mask.view(-1)
      
      # 4. Calculate the loss and mask it to remove the padding part
      pred = predictions.contiguous().view(-1,predictions.size(-1))#figure out the meaning later
      # print("pred2>>>",pred.shape)
      y_data_out1 = y_data_out.contiguous().view(-1)#.shape
      # print("y_data>>>",y_data_out1.shape)
      loss = criterion(pred, y_data_out1)

      masked_loss = torch.sum(loss*mask)
      
      # 5. Backward on the masked loss
      masked_loss.backward()
      
      # 6. Optional: Use torch.nn.utils.clip_grad_norm(model.parameters(), 2) to clip the gradient
      # torch.nn.utils.clip_grad_norm(model.parameters(), 2)

      # 7. Take a step with your optimizer
      optimizer.step()
      
      # 8. print the statistic (loss, edit distance and etc.) for analysis
      running_loss += float(masked_loss.item())/int(torch.sum(mask).item())

      #if  batch % 20 == 0:
    print('Train_loss: ', running_loss/batch)
    # torch.cuda.empty_cache()
    return att

In [15]:
def convert(logits, eos_token, prediction):
  if prediction:
    probas = F.softmax(logits, dim=2)
    preds = torch.argmax(probas, dim=2)
  else:
    preds = logits
  pred_list = []

  for i in range(preds.size(0)):
    eos_idx = (preds[i] ==  eos_token).nonzero()

    eos_idx = (len(preds[i])-1) if eos_idx.nelement() == 0 else eos_idx[0]

    pred_list.append(preds[i, :eos_idx])

  return pred_list

In [16]:
def get_text(logits, vocabulary, prediction=False):
  eos_token = vocabulary.index('<eos>')
  pred_list = convert(logits, eos_token, prediction)

  out_str = transform_index_to_letter(pred_list)
  return out_str

In [17]:
def calc_ED(prediction, target):
  for i, pred in enumerate(prediction):
    dist = []
    if i < len(target):
      dist = Levenshtein.distance(pred, target[i])
  return np.mean(dist)

In [18]:
def val(model, valid_loader, epoch):
    model.eval()
    running_loss = 0
    edit_distance = 0
    
    # 0) Iterate through your data loader
    for batch, (X, y) in enumerate(valid_loader):
      # print("X",np.array(X[0]).shape)
      # print("y",np.array(y[0]).shape)
      # optimizer.zero_grad()
      # 1) Set the inputs to the device.
      X_data, X_len = X
      X_data = X_data.to(device)
      y_data, y_len = y
      y_data = y_data.to(device)
      y_data_in = y_data[:,:-1]
      y_data_out = y_data[:,1:]

      # 2) Pass your inputs, and length of speech into the model.
      predictions, att = model(X_data, X_len, y_data, mode='test')

      predicted_text = get_text(predictions.data.cpu(), LETTER_LIST, True)
      target_text = get_text(y_data_out.data.cpu(), LETTER_LIST)
      if epoch%5 == 0:
        print("TARGET:", target_text)
        print("PREDICTED:", predicted_text)
      
      edit_distance += calc_ED(predicted_text, target_text)

      torch.nn.utils.clip_grad_norm(model.parameters(), 2)
    
    edit_distance /= len(valid_loader)
    print('Distance: ', edit_distance)
    torch.save(model.state_dict(), f"{save_location}Trial1Epoch{epoch}.pth")
    return edit_distance

In [19]:
# # TODO: Define your model and put it on the device here
# # ...
model = Seq2Seq(input_dim=40, vocab_size=len(LETTER_LIST), encoder_hidden_dim=512, 
                decoder_hidden_dim=512, embed_dim=512, key_value_size=128)
model = model.to(device)

In [23]:
state_dict = torch.load(f'{save_location}Trial1Epoch20.pth')
model.load_state_dict(state_dict)

<All keys matched successfully>

In [ ]:
n_epochs = 20
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss(reduction='none').to(device)
mode = 'train'

for epoch in range(21, 30):#Try with another 10 epochs
  print(f"Epoch {epoch}...")
  start_time = time.time()
  
  att = train(model, train_loader, criterion, optimizer, mode)
  plot_attention(att)
  
  train_time = time.time()
  print(f"Train time: {(train_time - start_time)/60} min")

  val(model, valid_loader, epoch)
  
  val_time = time.time()
  print(f"Val time: {(val_time - train_time)/60} min")

Epoch 21...


#### Test set prediction

In [ ]:
test_data = np.load('test.npy', allow_pickle=True, encoding='bytes')

In [ ]:
test_dataset = MyDataset(test_data)

In [ ]:
test_loader = data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, 
                               num_workers=4, pin_memory=True, collate_fn=collate_test)

In [ ]:
def test(model, test_loader):
    words = []
    model.to(device)
    start = time.time()
    #loss= 0
    cumm_loss =0.0
    # 1) Iterate through your loader
    for i,(X) in enumerate(test_loader):
      optimizer.zero_grad()
      # 2) Use torch.autograd.set_detect_anomaly(True) to get notices about gradient explosion
      torch.autograd.set_detect_anomaly(True)
      # 3) Set the inputs to the device.
      X_data, X_length = (X) #1344, 64, 40
      #print(target_data.shape)
      sequence_length = X_data.shape[2]
      X_data = X_data.to(device)
      X_length =  torch.LongTensor(X_length).to(torch.int)
      #print("shape input",input_data.shape,input_length.shape)
  

      # 4) Pass your inputs, and length of speech into the model.
      predicted,att = model(X_data,X_length, mode='test')
      #print('pre',predicted.shape)
      predicted_words = get_text(predicted.data.cpu(), LETTER_LIST,True)
      words.append(predicted_words)
    words = np.concatenate(words)
    return words

In [ ]:
final_prediction = test(model,test_loader)

In [ ]:
import pandas as pd
ids = np.arange(len(final_prediction))
out_dict = {"id":ids,"label":final_prediction}
df = pd.DataFrame(out_dict)
df.to_csv(f"{save_location}submission2.csv", index=False)

In [ ]:
df.head()

In [ ]:
!kaggle competitions submit -c 11785-homework-4-part-2-las-slack -f {save_location}submission1.csv -m "First attempt"